In [ ]:
import os
import shutil
import cv2
import sys
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
from tqdm import tqdm

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
#Needs work
#referenced - https://medium.com/@myringoleMLGOD/simple-convolutional-neural-network-cnn-for-dummies-in-pytorch-a-step-by-step-guide-6f4109f6df80
#	          - https://medium.com/@kritiikaaa/convolution-neural-networks-guide-for-your-first-cnn-project-7ea56f7f6960
# Formal dataset definition

# !ls -l '/content/drive/MyDrive/ECS_170/data/processed/merged_data'

#Add thing

#CNN definition
class plantCNN(nn.Module):
	def __init__(self):
		super(plantCNN, self).__init__()
		self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
		self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
		self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

		self.pool = nn.MaxPool2d(2, 2)
		self.relu = nn.ReLU()
		self.flatten = nn.Flatten()

		self.fc1 = nn.Linear(64 * 8 * 8, 128)
		self.fc2 = nn.Linear(128, 10) #Healthy, Bacterial spot, Early blight, Late blight, Leaf Mold, Mosaic virus, Septoria leaf spot, Spider mites, Target Spot, Yellow Leaf Curl Virus

	def forward(self, x):
		x = self.pool(self.relu(self.conv1(x)))
		x = self.pool(self.relu(self.conv2(x)))
		x = self.pool(self.relu(self.conv3(x)))
		x = self.flatten(x)
		x = self.relu(self.fc1(x))
		x = self.fc2(x)
		return x

# Load the dataset
data_root = "/content/drive/MyDrive/ECS_170/Tomato_Dataset_Splitted_V2"

transform = transforms.Compose([
     transforms.ToTensor()
])

train_data = dataset.ImageFolder(os.path.join(data_root, "train"), transform)
val_data = dataset.ImageFolder(os.path.join(data_root, "val"), transform)
test_data = dataset.ImageFolder(os.path.join(data_root, "test"), transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(val_data, batch_size=64, shuffle=False)

# Set up training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = plantCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training and validation loops
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for imgs, labels in tqdm(loader, desc="Training"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()
    acc = 100 * correct / total
    return running_loss / len(loader), acc

def eval_epoch(model, loader, criterion):
    model.eval()
    # store labels and predictions
    labels_array = []
    preds_array = []
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            labels_array.append(labels.cpu().numpy())
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            preds_array.append(preds.cpu().numpy())
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    acc = 100 * correct / total
    y_actual = np.concatenate(labels_array)
    y_predicted = np.concatenate(preds_array)
    return running_loss / len(loader), acc, y_actual, y_predicted

epochs = 17
patience = 5
threshold = 0.1
best_val_loss = 1

for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc, y_actual, y_predicted = eval_epoch(model, val_loader, criterion)
    print(f"\nEpoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%\n")
    print(confusion_matrix(y_actual, y_predicted))
    print(classification_report(y_actual, y_predicted))
    # early stopping
    patience_count = 0

    if(best_val_loss >= val_loss):
        bset_val_loss = val_loss

    if(val_loss > best_val_loss - threshold):
        patience_count += 1
    else:
        patience_count = 0

    if(patience_count > patience):
        break

os.makedirs("/content/drive/MyDrive/ECS_170/models", exist_ok=True)
torch.save(model.state_dict(), "/content/drive/MyDrive/ECS_170/models/plant_health_cnn.pth")

